<a href="https://colab.research.google.com/github/OpetherMB/grand-debat/blob/main/topic_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import libraries

In [1]:
import pandas as pd
import numpy as np
import nltk
from pprint import pprint
import pickle


nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import  word_tokenize


# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.phrases import Phrases, Phraser
from gensim.models import LsiModel
from gensim.models import CoherenceModel,HdpModel


# spacy for lemmatization
import spacy


# Plotting tools
!pip install pyLDAvis

import pyLDAvis
import pyLDAvis.gensim 
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
pyLDAvis.enable_notebook()

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


import re
import os
pd.set_option('max_colwidth', 400)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
     |████████████████████████████████| 1.6MB 5.3MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=68da68d304d53ce1f3767ea9d4a5de6022359aa07f1402c61617641338579680
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Colab Notebooks/grand debat
!pwd

/content/drive/MyDrive/Colab Notebooks/grand debat
/content/drive/MyDrive/Colab Notebooks/grand debat


In [4]:
# local lib

import util_topicModelling as  tpc

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Prepare data already cleaned and grouped

In [5]:
cleaned_data = pd.read_csv('clean_environement.csv', low_memory=False, dtype = str)
cleaned_data.fillna('')
cleaned_data.head()

,Unnamed: 0,authorZipCode,title,problems,reponse_probleme
0,0,97231,transition ecologique,NaN,NaN
1,1,57000,surpopulation,problemes auxquels trouve confronte ensemble planete denoncent parfait desordre gilets jaunes france dus avant surpopulation mondiale cette population passee milliards habitants milliards montera bientot milliards vers progres communication village mondial chaque individu fin fond asie fin fond afrique passant quartiers campagnes pays aspire vivre peu bl mer mal lotis concitoyens logement nour...,problemes auxquels trouve confronte ensemble planete denoncent parfait desordre gilets jaunes france dus avant surpopulation mondiale cette population passee milliards habitants milliards montera bientot milliards vers progres communication village mondial planete chaque individu fin fond asie fin fond afrique passant quartiers campagnes pays aspire vivre peu bl mer mal lotis concitoyens logem...
2,2,34140,climat,dereglements climatiques crue secheresse,eviter inondations obliger riverains communes nettoyer secteur
3,3,17400,pollution air eau,pollution air,taxer gros pollueurs entreprises bateaux marchands routiers etc
4,4,35430,economie vs ecologie,biodiversite disparition certaines especes,changer mode vie impulser nouvelle economie agricole


## Topic Modeling with Title column

### Build Model with non grouped data

In [6]:
list(cleaned_data['title'][2470:2475])

['vraie transition ecologique',
 'energies alternatives',
 'voiture avion paquebots portes contenairs',
 'mobilite',
 nan]

In [7]:
cleaned_data.fillna('',inplace=True)
count_nan = len(cleaned_data['title']) - cleaned_data['title'].count()
count_nan

0

# Using LDA

lets create a simple topic model with 8 topics using  Latent Dirichlet Allocation

### LDA with title column

In [8]:
data = list(cleaned_data['title'])
lda_model, corpus, id2word = tpc.get_topics(data, n_topics=8)

[(0,
  '0.056*"transports" + 0.031*"developper" + 0.030*"vie" + 0.029*"enfants" + '
  '0.029*"energies" + 0.027*"avenir" + 0.025*"taxe" + 0.020*"terre" + '
  '0.019*"carbone" + 0.015*"ecologiques"'),
 (1,
  '0.167*"ecologie" + 0.034*"taxer" + 0.026*"etre" + 0.025*"pollueurs" + '
  '0.023*"economie" + 0.014*"responsable" + 0.014*"citoyen" + '
  '0.012*"deplacements" + 0.012*"modele" + 0.011*"rendre"'),
 (2,
  '0.029*"agir" + 0.026*"climat" + 0.013*"temps" + 0.013*"ferroutage" + '
  '0.013*"toutes" + 0.013*"grand" + 0.013*"societe" + 0.013*"sante" + '
  '0.012*"conscience" + 0.012*"oui"'),
 (3,
  '0.041*"urgence" + 0.032*"climatique" + 0.029*"priorite" + 0.028*"produits" '
  '+ 0.022*"electrique" + 0.020*"contribution" + 0.019*"economique" + '
  '0.015*"voiture" + 0.013*"rechauffement" + 0.013*"mettre"'),
 (4,
  '0.064*"planete" + 0.035*"consommation" + 0.025*"reduire" + '
  '0.023*"vehicules" + 0.018*"limiter" + 0.017*"interdire" + 0.017*"sauver" + '
  '0.016*"education" + 0.014*"preser

In [9]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.397505  0.012609       1        1  18.190464
7      0.071201  0.375431       2        1  13.545730
5      0.064433 -0.115465       3        1  12.025206
0      0.053620 -0.081775       4        1  11.816066
3      0.047069 -0.055849       5        1  11.539911
1      0.049985 -0.038548       6        1  11.332338
2      0.054443 -0.057379       7        1  10.962780
4      0.056754 -0.039023       8        1  10.587505, topic_info=               Term         Freq        Total Category  logprob  loglift
7          ecologie  6134.000000  6134.000000  Default  30.0000  30.0000
0        ecologique  6305.000000  6305.000000  Default  29.0000  29.0000
1        transition  6022.000000  6022.000000  Default  28.0000  28.0000
27    environnement  3380.000000  3380.000000  Default  27.0000  27.0000
6         pollution  2528.000000  2528.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
478      polluantes   228.650564   229.484018   Topic8  -5.0130   2.2419
347           verte   223.057364   223.890559   Topic8  -5.0378   2.2418
1023     reellement   222.312627   223.148231   Topic8  -5.0412   2.2417
662     industriels   221.457727   222.290935   Topic8  -5.0450   2.2417
1779          heure   219.463404   220.296932   Topic8  -5.0541   2.2417

[271 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
860       2  0.996639    accelerer
866       3  0.998308      actions
1852      8  0.996902      affaire
480       7  0.999013         agir
88        3  0.998911  agriculture
...     ...       ...          ...
761       5  0.998320      voiture
781       5  0.998292     voitures
1199      6  0.995691         vrai
173       6  0.995111     vraiment
690       8  0.998713        vrais

[241 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 8, 6, 1, 4, 2, 3, 5])

on peut voir les different cluster quir represente les topic et ainsi  la frequence elvee des terms ...  ecologie , pollution, envirenment

## Topic Modeling For Problems column

In [10]:
cleaned_problems = cleaned_data['problems']
cleaned_problems.fillna('',inplace=True)
lda_model, corpus, id2word = tpc.get_topics(cleaned_problems, n_topics=5)

[(0,
  '0.021*"problemes" + 0.018*"importants" + 0.014*"lies" + 0.012*"ressources" '
  '+ 0.011*"autres" + 0.007*"propositions" + 0.006*"produits" + 0.006*"points" '
  '+ 0.006*"naturelles" + 0.005*"planete"'),
 (1,
  '0.398*"pollution" + 0.359*"air" + 0.026*"eau" + 0.014*"sols" + '
  '0.009*"energie" + 0.007*"fois" + 0.006*"impossible" + 0.005*"degradation" + '
  '0.005*"cause" + 0.005*"urgence"'),
 (2,
  '0.252*"biodiversite" + 0.238*"disparition" + 0.235*"especes" + '
  '0.225*"certaines" + 0.000*"accelerant" + 0.000*"ruralite" + '
  '0.000*"entrepreneuriat" + 0.000*"obsolescence" + 0.000*"programmee" + '
  '0.000*"obsolesence"'),
 (3,
  '0.024*"climatique" + 0.020*"dechets" + 0.016*"probleme" + 0.016*"erosion" + '
  '0.016*"toutes" + 0.015*"environnement" + 0.014*"dereglement" + '
  '0.014*"littoral" + 0.013*"important" + 0.013*"etre"'),
 (4,
  '0.249*"climatiques" + 0.248*"dereglements" + 0.233*"secheresse" + '
  '0.233*"crue" + 0.000*"analyser" + 0.000*"tremblements" + 0.000*"dis

In [11]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.197895  0.042602       1        1  24.505881
4      0.333739  0.265998       2        1  23.573062
3     -0.212942  0.048516       3        1  22.139769
2      0.177917 -0.380345       4        1  17.310473
1     -0.100819  0.023229       5        1  12.470815, topic_info=              Term          Freq         Total Category  logprob  loglift
97       pollution  19404.000000  19404.000000  Default  30.0000  30.0000
96             air  17499.000000  17499.000000  Default  29.0000  29.0000
92     climatiques  22970.000000  22970.000000  Default  28.0000  28.0000
94    dereglements  22907.000000  22907.000000  Default  27.0000  27.0000
95      secheresse  21514.000000  21514.000000  Default  26.0000  26.0000
...            ...           ...           ...      ...      ...      ...
1597   generalisee     53.981306     54.821348   Topic5  -6.8065   2.0663
1187         douce     55.282663     56.143477   Topic5  -6.7827   2.0663
37             eau   1278.392556   1667.536233   Topic5  -3.6418   1.8160
145           sols    676.080210   1015.508836   Topic5  -4.2788   1.6749
86          terres    152.097386    306.314766   Topic5  -5.7706   1.3817

[245 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
7498      3  0.165515  accelerant
7498      4  0.827576  accelerant
812       5  0.987187   activites
810       5  0.997468   agricoles
3008      1  0.496590  agrochimie
...     ...       ...         ...
343       1  0.999169        tres
995       1  0.998640         uns
771       5  0.997000     urgence
696       5  0.990174      urgent
202       3  0.998600         vie

[336 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 4, 3, 2])

La colonne des problèmes a donné de meilleurs résultats que le title qui semble logique puisque c'est celui qui contient le plus probablement des problèmes.

## Topic Modeling For Problems response column



In [12]:
data = list(cleaned_data['reponse_probleme'])
lda_model, corpus, id2word = tpc.get_topics(data, n_topics=5)

[(0,
  '0.011*"monde" + 0.010*"planete" + 0.010*"animaux" + 0.009*"nature" + '
  '0.008*"o" + 0.008*"especes" + 0.008*"autre" + 0.007*"aides" + 0.007*"hui" + '
  '0.007*"aujourd"'),
 (1,
  '0.018*"eau" + 0.016*"vie" + 0.011*"ville" + 0.010*"travail" + 0.009*"mieux" '
  '+ 0.007*"enfants" + 0.007*"qualite" + 0.006*"projets" + 0.006*"chaque" + '
  '0.005*"loi"'),
 (2,
  '0.018*"etre" + 0.011*"france" + 0.011*"pays" + 0.010*"ecologique" + '
  '0.008*"transition" + 0.008*"etat" + 0.008*"niveau" + 0.007*"cette" + '
  '0.007*"politique" + 0.006*"meme"'),
 (3,
  '0.026*"produits" + 0.016*"interdire" + 0.016*"consommation" + '
  '0.015*"limiter" + 0.015*"agriculture" + 0.015*"favoriser" + '
  '0.014*"energies" + 0.013*"production" + 0.013*"reduire" + 0.012*"arreter"'),
 (4,
  '0.024*"transports" + 0.018*"pollution" + 0.017*"transport" + 0.017*"taxer" '
  '+ 0.016*"entreprises" + 0.013*"developper" + 0.013*"vehicules" + '
  '0.012*"commun" + 0.011*"voiture" + 0.010*"villes"')]

Perplexity:  -8.

In [13]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.025462  0.176311       1        1  29.489643
3     -0.286922 -0.223465       2        1  21.057564
4     -0.110789  0.110920       3        1  20.728302
0      0.131593  0.179678       4        1  16.291218
1      0.291581 -0.243443       5        1  12.433273, topic_info=              Term          Freq         Total Category  logprob  loglift
458       produits  13670.000000  13670.000000  Default  30.0000  30.0000
241     transports  12366.000000  12366.000000  Default  29.0000  29.0000
377      pollution   9021.000000   9021.000000  Default  28.0000  28.0000
240      transport   8687.000000   8687.000000  Default  27.0000  27.0000
105          taxer   8562.000000   8562.000000  Default  26.0000  26.0000
...            ...           ...           ...      ...      ...      ...
16          chaque   1861.335295   4782.466960   Topic5  -5.1110   1.1411
120            ans   1512.858295   3288.032810   Topic5  -5.3183   1.3085
616      personnes   1350.264520   2565.171012   Topic5  -5.4320   1.4431
158          entre   1222.171475   2798.486192   Topic5  -5.5317   1.2563
2428  construction   1125.329890   1750.726312   Topic5  -5.6142   1.6428

[229 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
1517      2  0.999563     agricoles
259       2  0.999685  agriculteurs
260       2  0.999940   agriculture
1411      2  0.857150         aider
1411      3  0.142588         aider
...     ...       ...           ...
520       3  0.999784        villes
90        4  0.999437         vivre
445       3  0.999928       voiture
446       3  0.999760      voitures
1682      2  0.999793         zones

[243 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 5, 1, 2])

Après avoir construit des sujets en utilisant différentes colonnes, la colonne "probleme" a donné les meilleurs résultats et les sujets les plus compréhensibles. De plus, elle présentait le meilleur facteur de cohérence.

### NOTE : ce qui serais interessant c'est de faire une comparaison lda avec la version clean et version lemmetized ... à faire dans le future